In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [2]:
df = pd.read_csv("DataResults/phases.csv")
df = df.iloc[: , 1:]
df

,PID,Starting Interruption,Starting Task,Condition,Control,d_age,d_gender,d_education,d_asd,d_colorblind,...,te_p_count,te_p_correctness,te_p_time,te_p_times,te_p_percentage,te_p_percentage100,te_p_resumption,te_p_resumptions,te_p_interruptions,te_p_movestotal
0,150657-543833,1,1,1,1,36,Female,University,False,False,...,16,13.25,15.582868,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.828125,0.8125,17.367742,"[10.3176, 19.26643, 14.28441, 15.3505, 8.26959...",NaN,NaN
1,189842-198434,1,1,1,1,25,Female,University,False,False,...,16,16.00,9.153442,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",1.000000,1.0000,9.153442,"[8.934598, 8.7303, 15.23631, 10.87219, 6.22189...",NaN,NaN
2,285256-429012,1,1,1,1,27,Male,High School,False,False,...,16,15.50,6.216273,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.968750,0.9375,6.633005,"[5.776597, 13.51637, 6.810495, 5.625495, 5.892...",NaN,NaN
3,486637-941302,1,1,1,1,27,Male,University,False,False,...,16,15.50,8.996853,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.968750,0.9375,9.272375,"[7.770398, 11.45151, 7.735799, 6.586198, 10.07...",NaN,NaN
4,522111-792046,1,1,1,1,24,Male,Masters,False,False,...,16,10.25,10.141496,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.640625,0.4375,10.067041,"[12.67211, 15.00682, 12.10811, 9.307809, 10.55...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,883881-672590,2,2,2,0,19,Male,High School,False,False,...,16,NaN,242.962203,15.18513770625,NaN,NaN,1.070318,"[2.0045, 1.3836, 0.6834001, 0.9342001, 0.951, ...",227.0,14.1875
253,901440-351171,2,2,2,0,19,Female,High School,False,False,...,16,NaN,366.414080,22.900880000000008,NaN,NaN,1.246306,"[2.604299, 1.3343, 0.7325999, 0.6843, 0.715799...",294.0,18.3750
254,913584-192754,2,2,2,0,25,Female,Masters,False,False,...,16,NaN,468.229890,29.26436812500001,NaN,NaN,1.984025,"[4.927498, 2.3661, 1.5031, 1.5705, 1.5157, 1.1...",236.0,14.7500
255,967476-330431,2,2,2,0,25,Male,University,False,False,...,16,NaN,371.182687,23.19891793124999,NaN,NaN,1.734499,"[6.857299, 2.711, 1.1996, 1.0274, 1.2185, 1.57...",214.0,13.3750


In [3]:
df = pd.read_csv("DataResults/phases.csv")
df_conditions = df.iloc[: , 1:][["Condition", "Starting Interruption", "Starting Task", "Control"]]
df_conditions = df_conditions.drop_duplicates()

starting_interruption = []
starting_task = []
conditions = []
controls = []
phases = []
metric_description = []
task_description = []
t_exp = []
p_exp = []
difference_exp = []
#t_control = []
#p_control = []
#difference_control = []
#t_condition = []
#p_condition = []
# entry = [] # condition, metric, task, t-exp, p-exp , difference-exp, t-control, p-control, difference-control

In [4]:
df_conditions

,Condition,Starting Interruption,Starting Task,Control
0,1,1,1,1
15,2,1,1,1
30,1,1,2,1
45,2,1,2,1
60,1,2,1,1
75,2,2,1,1
92,1,2,2,1
108,2,2,2,1
124,1,1,1,0
139,2,1,1,0


In [5]:
# helpers for interpreting t-test statistics

def is_improvement(change):
    if (change > 0):
        return "increased [+]"
    return "decreased [-]"

def is_significant(p):
    if (p <= 0.005):
        return "extremely significantly [**]"
    elif (p <= 0.05):
        return "significantly [*]"
    elif (p <= 0.03):
        return "slightly"
    return ""

def compute_improvement(col1, col2):
    col3 = col1 - col2
    avg = sum(col3)/len(col3)
    t, p = stats.ttest_ind(col1, col2)
    return t, p, avg

In [6]:
def compare_improvement(task, interruption, condition, control, phase, metric_descript, df_data, pre_metric, post_metric):
    t_0, p_0, diff_0 = compute_improvement(
        df_data[(df_data['Starting Task'] == task) & (df_data['Starting Interruption'] == interruption) & (df_data['Condition'] == condition) & (df_data['Control'] == control)][pre_metric].astype(float), 
        df_data[(df_data['Starting Task'] == task) & (df_data['Starting Interruption'] == interruption) & (df_data['Condition'] == condition) & (df_data['Control'] == control)][post_metric].astype(float))
    
    print(t_0, p_0, diff_0)
    starting_interruption.append(interruption)
    starting_task.append(task)
    conditions.append(condition)
    controls.append(control)
    phases.append(phase)
    metric_description.append(metric_descript)
    t_exp.append(t_0)
    p_exp.append(p_0)
    difference_exp.append(diff_0)

In [8]:
for index, row in df_conditions.iterrows():
    if row['Starting Task'] == 1:
        compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'accuracy on primary task', df, 'a_p_pre_correctness', 'a_p_post_correctness')
    else:
        compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'accuracy on primary task', df, 'a_p_pre_movestotal', 'a_p_post_movestotal')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'accuracy on interrupting task', df, 'a_i_pre_percentage', 'a_i_post_percentage')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'speed on primary task', df, 'a_p_pre_time', 'a_p_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'interruption lag / speed on interrupting task', df, 'a_i_pre_time', 'a_i_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], row['Condition'], row['Control'], 'assessment', 'resumption lag', df, 'a_p_pre_resumption', 'a_p_post_resumption')

-0.42189753236077654 0.6763215854020874 -0.2
-0.422457400168947 0.6759177628589312 -0.025
-0.0015259490793487966 0.9987932894151859 -0.0019004117460320084
4.043579451794592 0.0003738259285793665 0.9920459699999999
0.29653166191610847 0.769012563863283 0.31540356444444534
-0.3744779337577218 0.710869994148526 -0.2
0.8143450710459562 0.4223194951887398 0.05
1.7244897834615058 0.09564429890108272 1.350502105555556
0.5209358550424876 0.6065071751248797 0.6440325282500001
2.1364006639242112 0.04152674056224395 2.3893057833333335
4.122186798883729 0.00030256317301559577 52.46666666666667
0.36689969285267504 0.716452156200321 0.008333333333333333
1.8302016112244328 0.07788487405659822 6.0681639208333324
2.4849762734762897 0.019201006633964436 0.6273464966666668
1.6692403624648617 0.10621471538964278 0.8349019333333334
3.250485081354487 0.0029957600071797504 75.4
0.7991784102357017 0.430914504288299 0.041666666666666664
2.1652203967671992 0.039044080995654394 5.399367904999997
1.55757782635833

In [9]:
columnTitles = {
    "Starting Interruption": starting_interruption,
    "Starting Task": starting_task,
    "Condition": conditions,
    "Control": controls,
    "Phase": phases,
    "Metric": metric_description,
    "t": t_exp,
    "p": p_exp,
    "diff": difference_exp
}

dataframe = pd.DataFrame(columnTitles)
dataframe.to_csv('phases.csv')
print('DONE')

DONE
